# Criando uma API Rest (Python/Django)

### 01. Configurando o ambiente

#### A. Criando a virtualenv

In [ ]:
# virtualenv <nome da venv>
virtualenv titanicenv

#### B. Ativando a virtualenv 

In [ ]:
titanicenv\Scripts\activate

#### C. Instalando os requisitos

In [ ]:
pip install -r requirements.txt

**Arquivo: requirements.txt**

`
Django==2.0.4
django-cors-headers==2.2.0
djangorestframework==3.8.2
psycopg2==2.7.4
pytz==2018.4`

### 02. Criando o projeto Django

In [ ]:
# django-admin startproject <nome do projeto> .
django-admin startproject config .

#### A árvore de arquivos deve estar:

`
/
   config/
      __init__.py
      settings.py
      urls.py
      wsgi.py
   titanicenv/
   manage.py
   requirements.txt
`

### 03. Configurações iniciais do projeto

#### A.  Banco de dados

Abra o arquivo **settings.py**, que está dentro da pasta config, e procure a lista **DATABASES**.

Ela estará assim:

```python
DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.sqlite3',
        'NAME': os.path.join(BASE_DIR, 'db.sqlite3'),
    }
}
```

Vamos mudá-la para:

```python
DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.postgresql_psycopg2',
        'NAME': 'xxx', # Nome do seu banco de dados
        'USER': 'xxx', # Nome do seu usuário do banco de dados
        'PASSWORD': 'xxx', # Senha do seu usuário do banco de dados
        'HOST': '127.0.0.1',
        'PORT': ''
    }
}
```

Assim, estamos configurando nosso projeto para usar o PostgreSQL.

Para testar a conexão com o banco, rodamos:

In [ ]:
python manage.py migrate

Esse comando cria no banco de dados as tabelas padrões do framework Django.
Se a conexão com o banco estiver ok, a saída no terminal será:

`
Operations to perform:
  Apply all migrations: admin, auth, contenttypes, sessions
Running migrations:
  Applying contenttypes.0001_initial... OK
  Applying auth.0001_initial... OK
  Applying admin.0001_initial... OK
  Applying admin.0002_logentry_remove_auto_add... OK
  Applying contenttypes.0002_remove_content_type_name... OK
  Applying auth.0002_alter_permission_name_max_length... OK
  Applying auth.0003_alter_user_email_max_length... OK
  Applying auth.0004_alter_user_username_opts... OK
  Applying auth.0005_alter_user_last_login_null... OK
  Applying auth.0006_require_contenttypes_0002... OK
  Applying auth.0007_alter_validators_add_error_messages... OK
  Applying auth.0008_alter_user_username_max_length... OK
  Applying auth.0009_alter_user_last_name_max_length... OK
  Applying sessions.0001_initial... OK
`

#### B. Configuração do Django Rest Framework

Ainda dentro do arquivo **settings.py**, procure a lista **INSTALLED_APPS**, de adicione no final da lista os dois seguintes apps:

```python
INSTALLED_APPS = [
    ...
    'corsheaders',
    'rest_framework',
]
```

No final do arquivo **settings.py**, adicione o seguinte código:

```python
REST_FRAMEWORK = {
    'DEFAULT_AUTHENTICATION_CLASSES': [],
    'DEFAULT_PERMISSION_CLASSES': [],
}

CORS_ORIGIN_ALLOW_ALL = True
```

Ainda no arquivo **settings.py**, troque a lista padrão **MIDDLEWARE** para a seguinte lista:

```python
MIDDLEWARE = [
    'corsheaders.middleware.CorsMiddleware',
    'django.middleware.common.CommonMiddleware',
    'django.middleware.security.SecurityMiddleware',
    'django.contrib.sessions.middleware.SessionMiddleware',
    'django.middleware.common.CommonMiddleware',
    'django.middleware.csrf.CsrfViewMiddleware',
    'django.contrib.auth.middleware.AuthenticationMiddleware',
    'django.contrib.messages.middleware.MessageMiddleware',
    'django.middleware.clickjacking.XFrameOptionsMiddleware',
]
```

### 04. Criação do app "titanic"

Crie uma pasta chamada "project" na raiz do projeto. Nesta pasta ficarão os apps do sistema. A árvore de arquivos ficará:

```
/
   config/
      __init__.py
      settings.py
      urls.py
      wsgi.py
   project/
   titanicenv/
   manage.py
   requirements.txt
```

Entre dentro da pasta project com o terminal:

```
cd  project
```

Dentro da pasta project, rode o seguinte comando para criar o app "titanic":

```python
python ../manage.py startapp titanic
```

A árvore de arquivos ficará:

```
/
   config/
      __init__.py
      settings.py
      urls.py
      wsgi.py
   project/
      titanic/
         migrations/
         __init__.py
         admin.py
         apps.py
         models.py
         tests.py
         views.py
   titanicenv/
   manage.py
   requirements.txt
```

Precisamos agora instalar nosso app. Volte até o arquivo **settings.py**, procure novamente a lista **INSTALLED_APPS**, e adicione o app titanic no final da lista, da seguinte forma:


```python
INSTALLED_APPS = [
    ...
    'project.titanic',
]
```

### 04. Criação da API "titanic": Models

Dentro da pasta **titanic**, abra o arquivo **models.py**. Substitua o conteúdo do arquivo pelo seguinte código:

In [ ]:
from django.db import models


class TitanicPeople(models.Model):

    passengerid = models.PositiveIntegerField(
        primary_key=True
    )
    survived = models.BooleanField(
        default=False
    )
    pclass = models.IntegerField(
        blank=True,
        null=True
    )
    name = models.CharField(
        max_length=100,
        blank=True,
        null=True
    )
    sex = models.CharField(
        max_length=6,
        blank=True,
        null=True
    )
    age = models.IntegerField(
        blank=True,
        null=True
    )
    Sibsp = models.IntegerField(
        blank=True,
        null=True
    )
    parch = models.IntegerField(
        blank=True,
        null=True
    )
    ticket = models.CharField(
        max_length=20,
        blank=True,
        null=True
    )
    fare = models.DecimalField(
        max_digits=12,
        decimal_places=4,
        blank=True, 
        null=True
    )
    cabin = models.CharField(
        max_length=20,
        blank=True,
        null=True
    )
    embarked = models.CharField(
        max_length=1,
        blank=True,
        null=True
    )


Em seguida, execute esses dois comandos (nesta ordem) para criar a tabela no banco de dados:

`
python manage.py makemigrations
python manage.py migrate
`

Repare que foi criada uma tabela chamada **titanic_titanicpeople** em seu banco de dados. 

### 05. Criação da API "titanic": Serializers

Dentro da pasta **titanic**, crie um arquivo chamado **serializers.py**. O conteúdo do arquivo deve ser o seguinte código:

In [ ]:
from rest_framework import serializers

from .models import TitanicPeople


class TitanicPeopleSerializer(serializers.ModelSerializer):

    class Meta:
        model = TitanicPeople
        fields = '__all__'

### 06. Criação da API "titanic": Views

Dentro da pasta **titanic**, abra o arquivo **views.py**. Substitua o conteúdo do arquivo pelo seguinte código:

In [ ]:
from rest_framework import viewsets

from .serializers import TitanicPeopleSerializer
from .models import TitanicPeople


class TitanicPeopleViewSet(viewsets.ModelViewSet):
    
    queryset = TitanicPeople.objects.all()
    serializer_class = TitanicPeopleSerializer


### 07. Criação da API "titanic": Rotas

Dentro da pasta **config**, abra o arquivo **urls.py**. Substitua o conteúdo do arquivo pelo seguinte código:

In [ ]:
from django.urls import path, include

from rest_framework import routers

from project.titanic.views import TitanicPeopleViewSet


router = routers.DefaultRouter()

router.register(r'titanic', TitanicPeopleViewSet)

urlpatterns = [
    path('', include('rest_framework.urls')),
    path('', include(router.urls)),
]


#### Para testar a API, execute:

```
python manage.py runserver
```